# 도구변수 추정
- 작성자: 고려대학교 경제학과 한치록 교수, 데이터사이언스팀 이창훈 과장

설명변수에 내생적(회귀식 오차항과 상관)인 변수가 포함되어 있을 때 OLS 추정량은 편향되고 이 편향은 표본크기가 아무리 커도 해소되지 않는다. 오히려 표본크기가 크면 클수록 잘못된 모수를 점점 더 분명하게 추정한다. 계량경제학에서는 이 경우 도구변수 추정법이 널리 사용된다. 도구변수 추정법이 작동하기 위해서는 최소한 내생적 설명변수 개수만큼의 추가 도구변수가 필요하다. 이 추가 도구변수들($Z$)은 외생적이어야 하고 내생적 설명변수들($X$)과 충분히 강하게 상관되어야 한다. 추가 도구변수의 선택은 연구자의 몫이다.

## Two-Stage Least Squares (2SLS)

도구변수 추정법 중 가장 널리 사용되는 것이 2단계 최소제곱법(2SLS)이다. 이 방법은 우선 내생적 설명변수들($X_2$)을 모든 외생변수들(외생적 설명변수들 $X_1$과 추가 도구변수들 $Z_2$)에 대하여 OLS 회귀한 다음 fitted values ($\hat{X}_2$)를 구하고(1단계 회귀), 그 다음 종속변수($y$)를 외생적 설명변수들($X_1$)과 이들 fitted values $\hat{X}_2$에 대하여 OLS 회귀하는 방법이다. 단, 둘째 단계 회귀에서 자동으로 도출되는 표준오차는 잘못이므로 제대로 된 표준오차를 따로 구하여야 한다.

2SLS는 [linearmodels][lm] 패키지의 `IV2SLS` 모듈에 구현되어 있다([statsmodels][sm] 패키지에도 기초적인 구현이 시도되어 있으나 0.15 버전에서는 사용할 수 있을 정도는 아니다). 그런데 [linearmodels][lm] 패키지의 수식 인터페이스는 절편을 자동으로 포함하지 않으므로 매우 불편하다. 이에 [linearmodels][lm] 패키지의 수식 인터페이스 부분을 `bok_da` 라이브러리에서 monkey patch하였다. monkey patch란 동적으로 특정 패키지에 코드를 덧붙이거나 기존 소스코드를 교체해서 원래의 동작을 사용자가 원하는 동작으로 변경하는 기법이다. `import bok_da`를 하면 패치가 적용된다. 원래 `IV2SLS`는 `robust` standard error 계산이 디폴트로 되어 있으나, monkey patch 시에 `unadjust`가 디폴트가 되도록 수정하였다.

2SLS는 다음 예제와 같이 하면 된다. 외생적 설명변수가 `exp76`과 `smsa76`이고 내생적 설명변수는 `ed76`, 추가 도구변수는 `momed`와 `daded`인 경우이다. 이는 formular에서 `[ed76 ~ momed+daded]`로 표현한다. 이 표현은 `ed76`을 내생변수로 보고, `momed`와 `daded`를 그에 대한 도구변수로 쓰겠다는 것을 의미한다. 아래에서 `import bok_da` 부분은 매우 중요하다. 이걸 해야 monkey patch가 이루어지기 때문이다.

[sm]: https://www.statsmodels.org/
[lm]: https://bashtage.github.io/linearmodels/

In [1]:
import bok_da
import pandas as pd
from linearmodels import IV2SLS

df = pd.read_csv('../data/Schooling.csv')
fm = 'lwage76 ~ [ed76 ~ momed+daded] + exp76 + smsa76'
mod = IV2SLS.from_formula(fm, data=df)
tsls = mod.fit()
print(tsls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                lwage76   R-squared:                      0.1705
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1696
No. Observations:                3010   F-statistic:                    359.02
Date:                Thu, May 22 2025   P-value (F-stat)                0.0000
Time:                        15:11:18   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                               Parameter Estimates                               
               Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------
Intercept         3.8318     0.1813     21.133     0.0000      3.4764      4.1871
exp76             0.0604     0.0049     

## 2SLS 관련 검정

### First-stage tests

도구변수가 내생적 설명변수와 충분히 상관되었는지를 보는 제1단계 검정(first-stage tests)은 다음과 같이 한다.

In [2]:
print(tsls.first_stage)

    First Stage Estimation Results    
                                  ed76
--------------------------------------
R-squared                       0.5000
Partial R-squared               0.1130
Shea's R-squared                0.1130
Partial F-statistic             191.64
P-value (Partial F-stat)      1.11e-16
Partial F-stat Distn         F(2,3005)
========================== ===========
Intercept                       13.412
                              (70.982)
exp76                          -0.3558
                             (-39.940)
smsa76[T.yes]                   0.4220
                              (5.4425)
momed                           0.1539
                              (11.044)
daded                           0.1111
                              (8.6693)
--------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


* 위에서 R-squared는 내생적 설명변수(`ed76`)을 외생적 설명변수들(`exp76`과 `smsa76`)과 추가 도구변수들(`momed`와 `daded`)에 대하여 회귀할 때의 R-squared이다. 이 값이 크다는 것은 내생적 설명변수에 대한 도구변수들의 설명력이 높다는 것이다. 한 가지 문제는 우리에게 필요한 것은 추가 도구변수emf의 설명력이 높아야 한다는 것인데, 추가 설명변수들의 기여는 낮으면서 외생적 설명변수들의 기여가 높아 R제곱이 클 수도 있다는 것이다. 그러므로 다른 지표들도 살펴보아야 한다.

* R-squared와 달리 Partial R-squared는 외생적 설명변수들을 통제한 상태에서 추가 도구변수들의 설명력을 의미한다. 이 수치는 내생적 설명변수(`ed76`)를 외생적 설명변수들에 대하여 회귀하여 잔차($\tilde{x}_2$라 하자)를 구하고, 추가적 도구변수들(`momed`와 `daded`) 각각을 외생적 설명변수들에 대하여 회귀하여 잔차($\tilde{z}_{2a}$와 $\tilde{z}_{2b}$라 하자)를 구한 후, $\tilde{x}_2$를 $\tilde{z}_{2a}$와 $\tilde{z}_{2b}$에 대하여 (절편 없이) 회귀할 때의 R제곱을 의미한다.

* Shea (1997)의 R-squared (partial R-squared)는 내생적 설명변수가 복수 개일 때까지 고려된 통계이다. 만약 내생적 설명변수가 1개이면 Shea의 partial R-squared는 partial R-squared와 동일하다.

* Partial F-statistic은 first-stage F 검정통계량으로서, 내생적 설명변수를 모든 외생변수들(외생적 설명변수와 추가 도구변수)에 대하여 OLS 회귀할 때 추가 도구변수들의 계수가 0이라는 귀무가설에 해당하는 검정통계량이다. 보통 이 값이 10보다 크면 도구변수가 강하다고 한다.

### Endogeneity test

내생적 설명변수가 1개일 때 제1단계 회귀로부터 얻은 잔차($\hat{v}_2$)를 회귀식에 포함시켜서 OLS 추정을 하고 $\hat{v}_2$의 계수가 0인지 검정함으로써 내생성 여부를 검정할 수 있다(귀무가설은 `ed76`이 외생적라는 것). 예를 들어 다음과 같이 해 보자.

In [3]:
from statsmodels.api import OLS

reg1 = OLS.from_formula('ed76 ~ exp76+smsa76+momed+daded', data=df).fit()
df['vhat'] = reg1.resid
reg2 = OLS.from_formula('lwage76 ~ ed76+exp76+smsa76 + vhat', data=df).fit()
print(reg2.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                lwage76   R-squared:                      0.2209
Model:                            OLS   Adj. R-squared:                 0.2198
No. Observations:                3010   F-statistic:                     213.0
Covariance Type:            nonrobust   Prob (F-statistic):             0.0000
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        3.83175    0.17587      21.79      0.000     3.48692     4.17658
smsa76[T.yes]    0.15590    0.01715       9.09      0.000     0.12228     0.18952
ed76             0.13450    0.01058      12.71      0.000     0.11375     0.15524
exp76            0.06041    0.00471      12.84      0.000     0.05119     0.06964
vhat            -0.05263    0.01123      -4.69      0.000    -0.07466    -0.03061

Notes:
1. Standard Errors assu

회귀 결과 마지막 줄에 `vhat`의 `t`값은 -4.69이고 p값이 0.000이므로 `ed76`이 외생적이라는 귀무가설은 기각된다. `linearmodels`에서 이는 fitting 후에 `.wooldridge_regression` 프로퍼티로써 구할 수 있다. 단, t검정이 아니라 카이제곱 검정을 한다.

In [4]:
tsls.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 21.9909
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x1cd8771ad50

위 결과에서 Statistic 값은 위의 t통계량을 제곱한 것과 (거의) 같고 p값도 (거의) 같다. 위 결과에 의하면 `ed76` 변수는 내생적이라는 결론을 얻는다.

### Overidentification test

추가적 도구변수들이 모두 외생적인지 검정해 보자. 이 검정은 추가 도구변수의 개수가 내생적 설명변수의 개수와 동일하면 실행할 수 없고, 추가 도구변수의 개수가 내생적 설명변수의 개수보다 더 크면(즉, overidentify되면) 실행할 수 있는 검정이므로 "overidentification test"라 한다. `linearmodels`에서는 다음 명령으로 검정을 수행할 수 있다.

In [5]:
tsls.wooldridge_overid

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 3.1853
P-value: 0.0743
Distributed: chi2(1)
WaldTestStatistic, id: 0x1cd8771b200

## Limited Information Maximum Likelihood

`LIML(Limited Information Maximum Likelihood)`은 단일 방정식만 보고 최대우도 원리로 내생변수를 보정하는 IV 추정법으로, 특히 도구변수가 약할 때(weak instruments) 2SLS보다 효율성과 편향 보정 측면에서 유리하다. 도구변수가 내생변수를 충분히 설명하지 못해 1단계 회귀에서 설명력이 약한 상황에서 LIML이 편향을 일부 조정해 줄 수 있다.

In [7]:
import bok_da
import pandas as pd
from linearmodels import IVLIML

df = pd.read_csv('../data/Schooling.csv')
fm = 'lwage76 ~ [ed76 ~ momed+daded] + exp76 + smsa76'
mod = IVLIML.from_formula(fm, data=df)
liml = mod.fit()
print(liml)

                          IV-LIML Estimation Summary                          
Dep. Variable:                lwage76   R-squared:                      0.1696
Estimator:                    IV-LIML   Adj. R-squared:                 0.1688
No. Observations:                3010   F-statistic:                    358.36
Date:                Thu, May 22 2025   P-value (F-stat)                0.0000
Time:                        15:20:41   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                               Parameter Estimates                               
               Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------
Intercept         3.8243     0.1822     20.992     0.0000      3.4672      4.1813
exp76             0.0606     0.0049     